<a href="https://colab.research.google.com/github/shinnew9/Apziva_practice_code/blob/main/Project3-PotentialTalents/grokai_api_w_lc_hf_space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Try different models (to get exposed to pprl LLMs):
- phi model from Microsoft (get exposed to pplr LLMs),
- qween model (from HuggingFace) can be done (from Transformer library).
! Stay on compute version of when using google Colab. (until next time, try on Instruct base model!)

2. Another thing to try:
- LLM framework called LangChain, I can make the model to be loaded using HuggingFace spaces, but still these are expensive (Maybe I can't use very large version; they'll make me to subscribe with high pay - check on it)

Next steps would be finetuning those LLMs and then RAG.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Necessary Package Installation

In [5]:
!pip install langchain-community

In [7]:
!pip install gradio

In [9]:
!pip install -q requests huggingface_hub pandas langchain gradio

### Open CSV

In [8]:
import numpy as np
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/data.csv')
df_copy = df.copy()
df_copy

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN
...,...,...,...,...,...
99,100,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,NaN
100,101,Human Resources Generalist at Loparex,"Raleigh-Durham, North Carolina Area",500+,NaN
101,102,Business Intelligence and Analytics at Travelers,Greater New York City Area,49,NaN
102,103,Always set them up for Success,Greater Los Angeles Area,500+,NaN


### GrokAi-API

In [14]:
from google.colab import userdata

grok_api_key = userdata.get('grokai_api')

# API_KEY = "xai-rqZTT2LaDt7rwrZQ56zQMXFAOr42Dn7Wa5MkqMcGGUHxN4Hya4vmKJXn8efAoogYo362ZngpgvVz5dNj"
# current_cell_code = get_ipython().get_ipython().user_ns['In'][-1]

# # Extract the API key from the cell code
# for line in current_cell_code.split('\n'):
#     if line.startswith(API_KEY):
#         grok_api_key = line.split(API_KEY)[1].strip()
#         break
print(grok_api_key)

xai-rqZTT2LaDt7rwrZQ56zQMXFAOr42Dn7Wa5MkqMcGGUHxN4Hya4vmKJXn8efAoogYo362ZngpgvVz5dNj


In [17]:
import json
import requests

from langchain.llms.base import LLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

class GrokLLM(LLM):
  """
  Custom LangChain LLM class to call Grok AI API.
  """
  model: str = "grok-1"

  # Added the _llm_type property
  @property
  def _llm_type(self) -> str:
      return "grok"

  def _call(self, prompt: str, stop=None, run_manager=None):
    api_url = "https://api.grok.com/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {grok_api_key}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": self.model,
        "messages": [{"role": "system", "content": prompt}],
        "temperature": 0.2,
        "max_tokens": 1000
    }

    response = requests.post(api_url, headers=headers, json=payload)
    response_json = response.json()

    if "choices" in response_json:
      return response_json["choices"][0]["message"]["content"].strip()

    return "Error: No response from Grok AI"

In [18]:
# Initialize Grok AI LLM
llm = GrokLLM()

In [19]:
# Define LangChain Prompt
prompt_template = PromptTemplate(
    input_variables=["search_term", "job_titles"],
    template="""Given the job titles of candidates and the search term,
    determine how fit these candidates are based on the similarity to the search term.
    Assign a fitness score between 0 to 1 based on their cosine similarity.
    Return the candidates' job titles in descending order based on the fitness score.

    Search term = {search_term}
    Job titles = {job_titles}"""
)

In [20]:
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

<ipython-input-20-4ad6eb42b6e8>:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt_template)


In [21]:
def get_fitness_scores_batch(job_titles, search_term):
    """
    Uses LangChain with Grok AI API to determine fitness scores for candidates.
    """
    try:
        response = llm_chain.run(search_term=search_term, job_titles=job_titles)

        # ✅ Extract job titles and scores
        ranked_candidates = []
        for line in response.split("\n"):
            parts = line.strip().split(":")
            if len(parts) == 2:
                try:
                    title, score = parts[0].strip(), float(parts[1].strip())
                    ranked_candidates.append((title, max(0, min(1, score))))  # Ensure scores are within [0,1]
                except ValueError:
                    continue  # Skip invalid lines

        return ranked_candidates

    except Exception as e:
        print(f"Error processing batch: {e}")
        return [(title, 0) for title in job_titles]  # Default to 0 if API call fails

In [23]:
# Process the Entire Dataset
search_term = "Aspiring Human Resources"
job_titles_list = df_copy["job_title"].tolist()

# Call LangChain with Grok AI API (Process all data at once)
ranked_results = get_fitness_scores_batch(job_titles_list, search_term)

# Map API results back to the dataset
fit_scores = {title: score for title, score in ranked_results}
df_copy["fit"] = df_copy["job_title"].map(fit_scores)

# Fix: Ensure 'fit' column has no NaN values
df_copy["fit"] = df_copy["fit"].fillna(0)

# Sort candidates by descending fit score
df_sorted = df_copy.sort_values(by="fit", ascending=False)
df_sorted.head(5)

Error processing batch: HTTPSConnectionPool(host='api.grok.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7e4c2f6b44d0>: Failed to resolve 'api.grok.com' ([Errno -2] Name or service not known)"))


,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,0
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,0
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,0
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,0


In [24]:
import os
import gradio as gr

iface = gr.Interface(
    fn = get_fitness_scores_batch,
    inputs = gr.Textbox(label="Enter Search Term (e.g., 'Aspiring Human Resources')"),
    outputs = gr.Dataframe(headers=["Job Title", "Fit Score"]),
    title = "Job Candidate Ranking",
    description = "Enter a search term to rank candidates based on similarity."
)

if __name__ == "__main__":
  iface.launch()

/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1024: UserWarning: Expected 2 arguments for function <function get_fitness_scores_batch at 0x7e4c2f7d6ac0>, received 1.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1028: UserWarning: Expected at least 2 arguments for function <function get_fitness_scores_batch at 0x7e4c2f7d6ac0>, received 1.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://54a73a274a3af61888.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
https://54a73a274a3af61888.gradio.live/